In [5]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df=pd.read_excel('cluster1.xlsx')
df.head()

,Unnamed: 0,InvoiceNo,StockCode,UnitPrice,Quantity,lower,Description,InvoiceDate,CustomerID,Country
0,2,536366,0.168142,1.85,6,hand warmer union jack,HAND WARMER UNION JACK,2010-12-01 08:28:00,1.000000,United Kingdom
1,3,536366,0.168129,1.85,6,hand warmer red polka dot,HAND WARMER RED POLKA DOT,2010-12-01 08:28:00,1.000000,United Kingdom
2,4,536367,0.169633,2.10,6,poppy's playhouse bedroom,POPPY'S PLAYHOUSE BEDROOM,2010-12-01 08:34:00,0.088096,United Kingdom
3,5,536367,0.169673,2.10,6,poppy's playhouse kitchen,POPPY'S PLAYHOUSE KITCHEN,2010-12-01 08:34:00,0.088096,United Kingdom
4,7,536367,0.163842,1.65,6,ivory knitted mug cosy,IVORY KNITTED MUG COSY,2010-12-01 08:34:00,0.088096,United Kingdom


In [6]:
df['Description']=df['Description'].str.strip()
df.dropna(axis=0,subset=['InvoiceNo'],inplace=True)
df['InvoiceNo']=df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]
df.head()

,Unnamed: 0,InvoiceNo,StockCode,UnitPrice,Quantity,lower,Description,InvoiceDate,CustomerID,Country
0,2,536366,0.168142,1.85,6,hand warmer union jack,HAND WARMER UNION JACK,2010-12-01 08:28:00,1.000000,United Kingdom
1,3,536366,0.168129,1.85,6,hand warmer red polka dot,HAND WARMER RED POLKA DOT,2010-12-01 08:28:00,1.000000,United Kingdom
2,4,536367,0.169633,2.10,6,poppy's playhouse bedroom,POPPY'S PLAYHOUSE BEDROOM,2010-12-01 08:34:00,0.088096,United Kingdom
3,5,536367,0.169673,2.10,6,poppy's playhouse kitchen,POPPY'S PLAYHOUSE KITCHEN,2010-12-01 08:34:00,0.088096,United Kingdom
4,7,536367,0.163842,1.65,6,ivory knitted mug cosy,IVORY KNITTED MUG COSY,2010-12-01 08:34:00,0.088096,United Kingdom


In [7]:
%%time
basket= (df[df['Country']=='United Kingdom']
         .groupby(['InvoiceNo','Description'])['Quantity']
         .sum().unstack().reset_index().fillna(0)
         .set_index('InvoiceNo'))



Wall time: 301 ms


In [8]:
basket

Description,HAND WARMER RED POLKA DOT,HAND WARMER UNION JACK,IVORY KNITTED MUG COSY,POPPY'S PLAYHOUSE BEDROOM,POPPY'S PLAYHOUSE KITCHEN
InvoiceNo,,,,,
536366,6.0,6.0,0.0,0.0,0.0
536367,0.0,0.0,6.0,6.0,6.0
536372,6.0,6.0,0.0,0.0,0.0


In [9]:
%%time
def encode_units(x):
    if x<=0:
        return 0
    if x>=1:
        return 1
basket_sets=basket.applymap(encode_units)


Wall time: 243 ms


In [10]:
basket_sets

Description,HAND WARMER RED POLKA DOT,HAND WARMER UNION JACK,IVORY KNITTED MUG COSY,POPPY'S PLAYHOUSE BEDROOM,POPPY'S PLAYHOUSE KITCHEN
InvoiceNo,,,,,
536366,1,1,0,0,0
536367,0,0,1,1,1
536372,1,1,0,0,0


In [11]:
import time
t = time.process_time()
frequent_itemsets=apriori(basket_sets,min_support=0.007,use_colnames=True)
rules=association_rules(frequent_itemsets,metric='lift',min_threshold=1)
elapsed_time = time.process_time() - t
print(elapsed_time)

0.015625


In [12]:
frequent_itemsets

,support,itemsets
0,0.666667,(HAND WARMER RED POLKA DOT)
1,0.666667,(HAND WARMER UNION JACK)
2,0.333333,(IVORY KNITTED MUG COSY)
3,0.333333,(POPPY'S PLAYHOUSE BEDROOM)
4,0.333333,(POPPY'S PLAYHOUSE KITCHEN)
5,0.666667,"(HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
6,0.333333,"(POPPY'S PLAYHOUSE BEDROOM, IVORY KNITTED MUG ..."
7,0.333333,"(POPPY'S PLAYHOUSE KITCHEN, IVORY KNITTED MUG ..."
8,0.333333,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ..."
9,0.333333,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ..."


In [13]:
rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(HAND WARMER UNION JACK),(HAND WARMER RED POLKA DOT),0.666667,0.666667,0.666667,1.0,1.5,0.222222,inf
1,(HAND WARMER RED POLKA DOT),(HAND WARMER UNION JACK),0.666667,0.666667,0.666667,1.0,1.5,0.222222,inf
2,(POPPY'S PLAYHOUSE BEDROOM),(IVORY KNITTED MUG COSY),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
3,(IVORY KNITTED MUG COSY),(POPPY'S PLAYHOUSE BEDROOM),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
4,(POPPY'S PLAYHOUSE KITCHEN),(IVORY KNITTED MUG COSY),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
5,(IVORY KNITTED MUG COSY),(POPPY'S PLAYHOUSE KITCHEN),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
6,(POPPY'S PLAYHOUSE KITCHEN),(POPPY'S PLAYHOUSE BEDROOM),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
7,(POPPY'S PLAYHOUSE BEDROOM),(POPPY'S PLAYHOUSE KITCHEN),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
8,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ...",(IVORY KNITTED MUG COSY),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
9,"(POPPY'S PLAYHOUSE KITCHEN, IVORY KNITTED MUG ...",(POPPY'S PLAYHOUSE BEDROOM),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf


In [14]:
frequent_itemsets

,support,itemsets
0,0.666667,(HAND WARMER RED POLKA DOT)
1,0.666667,(HAND WARMER UNION JACK)
2,0.333333,(IVORY KNITTED MUG COSY)
3,0.333333,(POPPY'S PLAYHOUSE BEDROOM)
4,0.333333,(POPPY'S PLAYHOUSE KITCHEN)
5,0.666667,"(HAND WARMER UNION JACK, HAND WARMER RED POLKA..."
6,0.333333,"(POPPY'S PLAYHOUSE BEDROOM, IVORY KNITTED MUG ..."
7,0.333333,"(POPPY'S PLAYHOUSE KITCHEN, IVORY KNITTED MUG ..."
8,0.333333,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ..."
9,0.333333,"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ..."


In [15]:
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,"(POPPY'S PLAYHOUSE KITCHEN, IVORY KNITTED MUG ...",(POPPY'S PLAYHOUSE BEDROOM),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
10,"(POPPY'S PLAYHOUSE BEDROOM, IVORY KNITTED MUG ...",(POPPY'S PLAYHOUSE KITCHEN),0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
11,(POPPY'S PLAYHOUSE KITCHEN),"(POPPY'S PLAYHOUSE BEDROOM, IVORY KNITTED MUG ...",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
12,(POPPY'S PLAYHOUSE BEDROOM),"(POPPY'S PLAYHOUSE KITCHEN, IVORY KNITTED MUG ...",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf
13,(IVORY KNITTED MUG COSY),"(POPPY'S PLAYHOUSE KITCHEN, POPPY'S PLAYHOUSE ...",0.333333,0.333333,0.333333,1.0,3.0,0.222222,inf


In [16]:

for i in range (0,len(rules)):
    print(list(rules.antecedents[i]),'->',list(rules.consequents[i]))


['HAND WARMER UNION JACK'] -> ['HAND WARMER RED POLKA DOT']
['HAND WARMER RED POLKA DOT'] -> ['HAND WARMER UNION JACK']
["POPPY'S PLAYHOUSE BEDROOM"] -> ['IVORY KNITTED MUG COSY']
['IVORY KNITTED MUG COSY'] -> ["POPPY'S PLAYHOUSE BEDROOM"]
["POPPY'S PLAYHOUSE KITCHEN"] -> ['IVORY KNITTED MUG COSY']
['IVORY KNITTED MUG COSY'] -> ["POPPY'S PLAYHOUSE KITCHEN"]
["POPPY'S PLAYHOUSE KITCHEN"] -> ["POPPY'S PLAYHOUSE BEDROOM"]
["POPPY'S PLAYHOUSE BEDROOM"] -> ["POPPY'S PLAYHOUSE KITCHEN"]
["POPPY'S PLAYHOUSE KITCHEN", "POPPY'S PLAYHOUSE BEDROOM"] -> ['IVORY KNITTED MUG COSY']
["POPPY'S PLAYHOUSE KITCHEN", 'IVORY KNITTED MUG COSY'] -> ["POPPY'S PLAYHOUSE BEDROOM"]
["POPPY'S PLAYHOUSE BEDROOM", 'IVORY KNITTED MUG COSY'] -> ["POPPY'S PLAYHOUSE KITCHEN"]
["POPPY'S PLAYHOUSE KITCHEN"] -> ["POPPY'S PLAYHOUSE BEDROOM", 'IVORY KNITTED MUG COSY']
["POPPY'S PLAYHOUSE BEDROOM"] -> ["POPPY'S PLAYHOUSE KITCHEN", 'IVORY KNITTED MUG COSY']
['IVORY KNITTED MUG COSY'] -> ["POPPY'S PLAYHOUSE KITCHEN", "POPPY'S 